In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score

#Przygotowanie danych
filename = 'cleaned_top_500.csv'
path = f"../data/02_interim/{filename}"
data = pd.read_csv(path)

# >70% = dobra gra
data['target_bin'] = (data['pct_pos_total'] > 70).astype(int)
data = data.dropna(subset=['target_bin'])

X = data.drop(columns=['pct_pos_total', 'target_bin'])
y = data['target_bin']

#tylko dane numeryczne
X = X.select_dtypes(include=[np.number])

#podział danych na treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

#Lista modeli
models = {
    "RandomForest": RandomForestClassifier(random_state=42)
}

#Ewaluacja
results = []

for name, model in models.items():
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]  # potrzebne do ROC-AUC

    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1": f1_score(y_test, y_pred),
        "ROC-AUC": roc_auc_score(y_test, y_proba)
    })

#Tabela wyników
results_df = pd.DataFrame(results).sort_values(by="F1", ascending=False)
print(results_df)

In [ ]:
# Metryka
# Dodatkowe importy
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

# Target
target = "pct_pos_total"

# y jako liczby (gdyby coś było typu string)
y = pd.to_numeric(data[target], errors="coerce")

# Wszystkie kolumny poza targetem -> zamiana tekstu na one-hot
X = data.drop(columns=[target])
X = pd.get_dummies(X, drop_first=True)

# Podział na zbiory testowy i trenujący
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Kolumny numeryczne i kategoryczne
num_cols = [c for c in X.columns if pd.api.types.is_numeric_dtype(X[c])]
cat_cols = [c for c in X.columns if c not in num_cols]

# Model
model = RandomForestRegressor(random_state=42, n_estimators=200)
model.fit(X_train, y_train)

# Predykcja
y_pred = model.predict(X_test)

# Metryki
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("\nWynik metryki RMSE:")
print(f"RMSE: {rmse:.3f}")